In [94]:
import pandas as pd, numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier as cart_clf
from sklearn.model_selection import train_test_split
from sklearn.tree._tree import TREE_LEAF
from sklearn.metrics import accuracy_score

data = pd.read_csv('spam.data.csv', delimiter = ' ', header = None)
X, y = data.iloc[:, :-1].values, data.iloc[:, -1:].values.flatten()

In [95]:
cart_model = cart_clf(random_state=0)

### Part b and c

In [96]:
def countLeafNodes(tree, root, count) :
    left = t.children_left[root]
    right = t.children_right[root]
    
    if left == right == TREE_LEAF :
        return count + 1
    elif left == TREE_LEAF != right:
        return countLeafNodes(t, right, count)
    elif left != TREE_LEAF == right:
        return countLeafNodes(t, left, count)
    else :
        return countLeafNodes(t, left, count) + countLeafNodes(t, right, count)
    

def pruneLeaves(tree, root, rem_depth) :
    if root == TREE_LEAF : return
    
    if rem_depth == 0 :
        tree.children_left[root] = tree.children_left[root] = TREE_LEAF
        return
    
    left = tree.children_left[root]
    right = tree.children_right[root]
    
    pruneLeaves(tree, left, rem_depth - 1)
    pruneLeaves(tree, right, rem_depth - 1)
    
def cross_validation(X, y, size ) :
    scores = []
    for i in range(size) :
        X_train, X_test, y_train, y_test = train_test_split( X, y, test_size= 1.0/size) #random selection

        cart_model.fit(X_train, y_train)

        y_pred = cart_model.predict(X_test)
        scores.append(accuracy_score(y_test,y_pred))
    np_score = np.hstack(scores)
    return 1.0 - np.mean(np_score)

In [97]:
#original
print ("original score {}".format(round((cross_validation(X, y, size = 10)),3)))
max_depth = cart_model.tree_.max_depth
min_score = 1.0
min_index = -1
for i in range(1, max_depth) :
    score = round(cross_validation(X, y, size = 10),3)
    (min_index, min_score) = (i, score) if score < min_score else (min_index, min_score)
    print("{} --> {}".format(i, score))

print("Optimal tree by pruning {} levels and score : {}".format (min_index, min_score))
    


original score 0.085
1 --> 0.086
2 --> 0.08
3 --> 0.092
4 --> 0.081
5 --> 0.085
6 --> 0.08
7 --> 0.085
8 --> 0.085
9 --> 0.084
10 --> 0.091
11 --> 0.089
12 --> 0.087
13 --> 0.082
14 --> 0.083
15 --> 0.085
16 --> 0.084
17 --> 0.084
18 --> 0.082
19 --> 0.085
20 --> 0.08
21 --> 0.084
22 --> 0.086
23 --> 0.081
24 --> 0.086
25 --> 0.086
26 --> 0.089
27 --> 0.086
Optimal tree by pruning 2 levels and score : 0.08
